In [9]:
# Alright lets see what this is all about
import numpy as np
import pandas as pd
import sklearn 

In [10]:
df = pd.read_csv('Kirk_All.csv')
df


,FID,FID_WA_Mai,TYPE,LENGTH,DIAMETER,MATERIAL,INSTYEAR,MUSYM,Dist_Rail,SLP_CLASS,FULL_STRNA,SPEED,FUNC_CLASS
0,0,1,FIRE,2.0,8.0,UNK,NaN,Ur,6000,2.0,120TH AVE NE,25,MINOR ARTERIAL
1,1,2,FIRE,2.0,2.0,UNK,1/1/1992 0:00:00,AgC,7000,1.0,122ND LN NE,25,NEIGHBORHOOD ACCESS
2,2,3,FIRE,1.0,0.0,UNK,NaN,AgC,6000,0.0,124TH AVE NE,35,PRINCIPAL ARTERIAL
3,3,4,FIRE,3.0,6.0,DI,1/1/2005 0:00:00,Ur,4000,0.0,LAKE WASHINGTON BLVD NE,35,PRINCIPAL ARTERIAL
4,4,5,FIRE,149.0,8.0,AC,1/1/1963 0:00:00,Ur,7000,6.0,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
41816,41816,3963,DIST,1.0,12.0,DI,1/1/1971 0:00:00,No,9000,0.0,120TH AVE NE,25,COLLECTOR
41817,41817,5622,DIST,1.0,8.0,DI,1/1/2003 0:00:00,AgC,10000,2.0,117TH AVE NE,25,NEIGHBORHOOD ACCESS
41818,41818,5623,DIST,1.0,8.0,DI,1/1/2003 0:00:00,AgC,10000,2.0,117TH AVE NE,25,NEIGHBORHOOD ACCESS
41819,41819,1093,DIST,83.0,12.0,DI,1/1/2009 0:00:00,AgC,5000,0.0,NE 115TH PL,35,COLLECTOR


In [11]:
grouping = df.groupby('FID_WA_Mai').groups
unique_pipes = df['FID_WA_Mai'].unique()
indices = []
for i in range(len(unique_pipes)):
    pipe = unique_pipes[i]
    val = grouping[pipe][0]
    indices.append(val)

In [17]:
filtered = df.loc[indices, 'FID_WA_Mai':'FUNC_CLASS']
filtered = filtered.drop('SLP_CLASS', axis=1)
filtered['SLOPE'] = slopes.values
filtered


,FID_WA_Mai,TYPE,LENGTH,DIAMETER,MATERIAL,INSTYEAR,MUSYM,Dist_Rail,FULL_STRNA,SPEED,FUNC_CLASS,SLOPE
0,1,FIRE,2.0,8.0,UNK,NaN,Ur,6000,120TH AVE NE,25,MINOR ARTERIAL,2.000000
1,2,FIRE,2.0,2.0,UNK,1/1/1992 0:00:00,AgC,7000,122ND LN NE,25,NEIGHBORHOOD ACCESS,1.000000
2,3,FIRE,1.0,0.0,UNK,NaN,AgC,6000,124TH AVE NE,35,PRINCIPAL ARTERIAL,0.000000
3,4,FIRE,3.0,6.0,DI,1/1/2005 0:00:00,Ur,4000,LAKE WASHINGTON BLVD NE,35,PRINCIPAL ARTERIAL,0.000000
4,5,FIRE,149.0,8.0,AC,1/1/1963 0:00:00,Ur,7000,,,,3.888889
...,...,...,...,...,...,...,...,...,...,...,...,...
41801,7742,DIST,15.0,8.0,DI,1/1/2017 0:00:00,AgD,5000,96TH AVE NE,25,NEIGHBORHOOD ACCESS,0.785714
41804,7743,DIST,296.0,8.0,AC,1/1/1943 0:00:00,AgC,9000,NE 68TH ST,25,MINOR ARTERIAL,4.000000
41816,3963,DIST,1.0,12.0,DI,1/1/1971 0:00:00,No,9000,120TH AVE NE,25,COLLECTOR,4.000000
41817,5622,DIST,1.0,8.0,DI,1/1/2003 0:00:00,AgC,10000,117TH AVE NE,25,NEIGHBORHOOD ACCESS,3.000000


In [31]:
idx = df['FULL_STRNA'] == '120TH AVE NE'
#filt = df[df['FULL_STRNA'].str.contains('7TH AVE')]
filt = df[(df['FULL_STRNA'] == '7TH AVE') & (df['DIAMETER'] == 6.0) & (df['MATERIAL'] == 'CI')]

date = filtered[(filtered['FULL_STRNA'].str.contains('1ST')) & (filtered['INSTYEAR'].str.contains('2004'))]
date

,FID_WA_Mai,TYPE,LENGTH,DIAMETER,MATERIAL,INSTYEAR,MUSYM,Dist_Rail,FULL_STRNA,SPEED,FUNC_CLASS,SLOPE
920,213,DIST,1.0,12.0,DI,1/1/2004 0:00:00,AmC,14000,1ST ST,25,NEIGHBORHOOD ACCESS,2.0
921,214,DIST,39.0,6.0,DI,1/1/2004 0:00:00,AmC,14000,1ST ST,25,NEIGHBORHOOD ACCESS,1.0
924,215,DIST,31.0,12.0,DI,1/1/2004 0:00:00,AmC,14000,1ST ST,25,NEIGHBORHOOD ACCESS,2.5
926,216,DIST,2.0,8.0,DI,1/1/2004 0:00:00,AmC,14000,1ST ST,25,NEIGHBORHOOD ACCESS,2.0
927,217,DIST,2.0,12.0,DI,1/1/2004 0:00:00,AmC,14000,1ST ST,25,NEIGHBORHOOD ACCESS,2.0
928,218,DIST,260.0,12.0,DI,1/1/2004 0:00:00,AmC,14000,1ST ST,25,NEIGHBORHOOD ACCESS,1.1
38391,6682,DIST,28.0,1.5,DI,1/1/2004 0:00:00,AmC,14000,1ST ST,25,NEIGHBORHOOD ACCESS,1.0


In [12]:
df = pd.read_csv('Kirk_All.csv')
slopes = df[['FID_WA_Mai', 'SLP_CLASS']]
slopes = slopes.groupby('FID_WA_Mai')['SLP_CLASS'].mean()
slopes = pd.Series(slopes)
slopes

FID_WA_Mai
1       2.000000
2       1.000000
3       0.000000
4       0.000000
5       3.888889
          ...   
7739    0.785714
7740    4.000000
7741    4.000000
7742    3.000000
7743    0.777778
Name: SLP_CLASS, Length: 7650, dtype: float64

In [23]:
df['INSTYEAR'].unique()

array([nan, '1/1/1992 0:00:00', '1/1/2005 0:00:00', '1/1/1963 0:00:00',
       '1/1/1978 0:00:00', '1/1/2002 0:00:00', '1/1/2000 0:00:00',
       '1/1/1990 0:00:00', '1/1/2007 0:00:00', '1/1/1988 0:00:00',
       '1/1/1997 0:00:00', '1/1/1993 0:00:00', '1/1/2009 0:00:00',
       '1/1/2003 0:00:00', '1/1/1999 0:00:00', '1/1/1987 0:00:00',
       '1/1/1996 0:00:00', '1/1/1998 0:00:00', '1/1/2004 0:00:00',
       '1/1/2006 0:00:00', '1/1/1991 0:00:00', '1/1/1980 0:00:00',
       '1/1/1989 0:00:00', '1/1/2010 0:00:00', '1/1/1944 0:00:00',
       '1/1/2001 0:00:00', '1/1/2008 0:00:00', '7/15/1964 0:00:00',
       '1/1/1983 0:00:00', '1/1/1968 0:00:00', '1/1/1971 0:00:00',
       '8/5/1960 0:00:00', '1/1/1970 0:00:00', '1/1/1960 0:00:00',
       '1/1/1943 0:00:00', '1/1/1995 0:00:00', '1/1/1975 0:00:00',
       '1/1/1961 0:00:00', '1/1/1979 0:00:00', '1/1/1972 0:00:00',
       '1/1/1953 0:00:00', '1/1/1994 0:00:00', '1/1/1982 0:00:00',
       '1/1/1976 0:00:00', '1/1/1967 0:00:00', '1/1/1981

In [3]:
# Get statistics about dataset 
num_pipes = df['FID_WA_Mai'].unique()
